# Final Notebook Pt 1: means2work, race/eth, income, nbhooods

**By:** Andrew Williams and Ben Brassette
    
**Description:** This notebook is building on earlier queries of means to work. Purpose is to intergrate and align race/eth and median income data. We are tacking points 1, 3, 4, 5, and 7 in this notebook.

**Expected Outputs**
* Trimmed data high transit areas--use same tracts for trimmed race/eth and median income data
* Descriptive Statistics: Bar graphs of race/eth and income by using high public transit data
* Side by side maps

**Areas Where We Need More Work**
* Spatial Statistics of means2work
* Overlay of rail stops and bus lines
* Descriptive Statistics of Access to Car
* Inclusion of commute time and access to car maps (by high transit query)
* Descriptive Statistics of Jobs
* Interactive Map using some combination of our datasets

**Notes for self to advance project:**
1. Biggest step forward, intergrating our notebooks in the remaining few weeks in a targeted fashion
2.  Overlay heavy rail and light rail stops
3.  Provide more accurate description of the areas through a spatial join
4. Compare with race/ethnicity and income data; we should create some side by side comparisons of transit use and race/eth and income
5. Query "high" transit tracts, cross reference with income and race/eth, potential to add spatial dimension of LA neighborhoods
6. Availability of cars in households
7. Bring a shaprer focus to Central LA and the San Fernando Valley

**TOC**
* Section 2: Library Imports
* Section 3: Mode of Travel
* Section 4: Race/Eth and Income
* Section 5: Public Transit Query
* Section 6: Attribute Merge
* Section 7: Matching Tracts
* Section 8: Graphs
* Section 9: Maps

# Library Import 


Importing various libaries

In [ ]:
import urllib.request, json 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

Libraries good to go. 

# Importing Final 1.1 Data: m2w_income_race

In [ ]:
m2w_income_race= gpd.read_file('m2w_income_race_new.geojson') 

In [ ]:
type(m2w_income_race)

In [ ]:
m2w_income_race.shape

In [ ]:
m2w_income_race.head(4)

In [ ]:
m2w_income_race.columns.to_list()

In [ ]:
m2w_income_race.columns=['geoid',
 'name',
 'Neighborhood',
 'Median Income',
 'Total Workers Commuting',
 'Car Total',
 'Drove alone',
 'Carpooled',
 'Public transportation',
 'Bus',
 'Subway or elevated il',
 'Long-distance train or commuter rail',
 'Light rail',
 'Worked from home',
 '%Car Total',
 '%Drove alone',
 '%Carpooled',
 '%Public transportation',
 '%Bus',
 '%Subway or elevated il',
 '%Long-distance train or commuter rail',
 '%Light rail',
 '%Worked from home',
 'Total Pop Race',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific Islander',
 'Hispanic or Latino',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Pacific Islander',
 '%Hispanic or Latino',
 'geometry',]

In [ ]:
m2w_income_race.sample(4)

# Exploring Neighborhoods

Purpose is to query nbhoods, define new variables, find missing tracts

## Lay of the Land

### Public Transit

In [ ]:
nbh_pt20 = m2w_income_race[m2w_income_race['%Public transportation']>20]


In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
pt_trim=nbh_pt20.sort_values(by='%Public transportation',ascending = False)
pt_trim

In [ ]:
pt_trim[['geoid','Neighborhood','%Public transportation']]

In [ ]:
pt_trim['Neighborhood'].value_counts()

First four neighborhoods easily make up the vast majority. All located in Central LA. Panaroma City snuck in with 4 census tracts. The rest are either South LA or more Central LA tracts.

In [ ]:
nbh_pt20_count=nbh_pt20['Neighborhood'].value_counts()

In [ ]:
nbh_pt20_count = nbh_pt20_count.reset_index()
nbh_pt20_count

### Bus

In [ ]:
nbh_bus20 = m2w_income_race[m2w_income_race['%Bus']>20]

In [ ]:
nbh_bus20.sample(5)

In [ ]:
nbh_bus20['Neighborhood'].value_counts()

### Subway

In [ ]:
nbh_sub5 = m2w_income_race[m2w_income_race['%Subway or elevated il']>5]

In [ ]:
nbh_sub5

In [ ]:
nbh_sub5['Neighborhood'].value_counts()

### Commuter

In [ ]:
nbh_commute3 = m2w_income_race[m2w_income_race['%Long-distance train or commuter rail']>3]

In [ ]:
nbh_commute3

In [ ]:
nbh_commute3['Neighborhood'].value_counts()

### Light Rail

In [ ]:
nbh_lr4 = m2w_income_race[m2w_income_race['%Light rail']>4]

In [ ]:
nbh_lr4

**Taken from the above value counts**

**Most Frequenct users of Public Transit:**
* Westlake                  23x
* Koreatown                 18x
* Pico-Union                13x
* East Hollywood             9x
* Historic South-Central     6x
* Downtown                   5x
* Hollywood                  5x
* Panorama City              4x
* Harvard Heights            3x
* Vermont Square             2x
* Boyle Heights              2x
* Central-Alameda            2x
* Exposition Park            2x
* Vermont Knolls             1x
* South Park                 1x
* Florence                   1x
* Echo Park                  1x
* University Park            1x
* Chinatown                  1x
* Adams-Normandie            1x
* Highland Park              1x
* Los Feliz                  1x


**Other Neighborhoods to conisder:**
* North Hollywood (sub)     4x
* East Hollywood (sub)      3x
* Mount Washington (sub)    1x
* Chinatown (sub)           1x
* Studio City  (sub)        1x
* Montecito Heights (sub)   1x
* Palms (sub)               1x
* Valley Village (sub)      1x
* Baldwin Hills/Crenshaw (distance rail)    2x
* Jefferson Park (distance rail)            1x
* Exposition Park   (distance rail)         1

A breakdown by region

**SF Valley**
* Studio City
* North Hollywood
* Panorama City 
* Valley Village 

 
**Westside**
* Palms

**Central LA**
* Hollywood
* East Hollywood 
* Koreatown   
* Westlake  
* Downtown
* Echo Park
* Chinatown
* Harvard Heights
* Pico-Union  
* Los Feliz  

**South LA**
* Historic South-Central
* Vermont Square  
* Vermont Knolls 
* Central-Alameda 
* Exposition Park
* South Park
* Florence   
* University Park
* Adams-Normandie
* Baldwin Hills/Crenshaw
* Jefferson Park
* Exposition Park

**Eastside**
* Boyle Heights  

**Northeast LA**
* Highland Park 
* Mount Washington 
* Montecito Heights 

# Visual Displays

## Prepping Data

Bring in more libraries!

In [ ]:
# for interactive plots
import plotly.express as px

# to import open data,
from sodapy import Socrata

import urllib.request, json 

from shapely.geometry import Polygon

Prep data and ensure it's ready to go.

In [ ]:
m2w_income_race.columns.to_list()

In [ ]:
#remove geometry
keep_df=['geoid',
 'name',
 'Neighborhood',
 'Median Income',
 'Total Workers Commuting',
 'Car Total',
 'Drove alone',
 'Carpooled',
 'Public transportation',
 'Bus',
 'Subway or elevated il',
 'Long-distance train or commuter rail',
 'Light rail',
 'Worked from home',
 '%Car Total',
 '%Drove alone',
 '%Carpooled',
 '%Public transportation',
 '%Bus',
 '%Subway or elevated il',
 '%Long-distance train or commuter rail',
 '%Light rail',
 '%Worked from home',
 'Total Pop Race',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific Islander',
 'Hispanic or Latino',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Pacific Islander',
 '%Hispanic or Latino',
]

In [ ]:
df = m2w_income_race [keep_df]

In [ ]:
type(df)

In [ ]:
df.sample(5)

In [ ]:
df.shape

Data is ready to go.

## Central LA

### Westlake trial run

In [ ]:
#westlake= df[df['%Public transportation']>20 &
             # (df[df.Neighborhood == 'Westlake'])               ]

In [ ]:
westlake= df.query("Neighborhood == 'Westlake'")

In [ ]:
westlake

In [ ]:
westlake.shape

In [ ]:
westlake.head()

In [ ]:
fig = px.histogram(df, 
                   x = "%Public transportation",
                   title='Percent Public Transportation Means to Work',
                   nbins=50, 
                   color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
fig = px.bar(westlake, x="geoid", y="%White", color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
fig = px.bar(westlake, x="geoid", y="%Black", color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
fig = px.bar(westlake, x="geoid", y="%White", color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
fig = px.bar(westlake, x="geoid", y="%Hispanic or Latino", color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = px.bar(westlake, 
             y="geoid", 
             x=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             orientation='h')
fig.show()

In [ ]:
fig = px.bar(westlake, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Westlake Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

Plotly Colors: https://plotly.com/python/discrete-color/

In [ ]:
fig = px.bar(westlake, 
             y='geoid', 
             x="Median Income",
             title='Westlake Median Income',
             color_discrete_sequence=['#00CC96'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

-facet ordering: neighborhood or by public transit, bus, subway,car, etc(4)
-graphs (5)

In [ ]:
fig = px.bar(westlake, x="geoid", y="%Public transportation", color_discrete_sequence=['#00CC96'])
fig.show()

In [ ]:
fig = px.bar(westlake, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Westlake Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(westlake, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Westlake Mode of Trasnportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
m2w_income_race[m2w_income_race['%Public transportation'] > 20].plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='cividis',
                 legend=True, 
                 scheme='NaturalBreaks',)

In [ ]:
m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='cividis',
                 legend=True, 
                 scheme='NaturalBreaks',)

Plot differet races altogether
Plot income and means2work
plot hispanic, asian, white and means2work

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Westlake Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='magma_r',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Downtown

In [ ]:
dtown= df.query("Neighborhood == 'Downtown'")

In [ ]:
dtown.shape

In [ ]:
type(dtown)

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Downtown Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(dtown, 
             y='geoid', 
             x="Median Income",
             title='Downtown Median Income',
             color_discrete_sequence=['#FFA15A'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Downtown Mode of Trasnportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Downtown Mode of Trasnportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Downtown Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Downtown'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='Purples_r',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Koreatown

In [ ]:
ktown= df.query("Neighborhood == 'Koreatown'")

In [ ]:
ktown.shape

In [ ]:
type(ktown)

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Koreatown Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(dtown, 
             y='geoid', 
             x="Median Income",
             title='Koreatown Median Income',
             color_discrete_sequence=['#FFA15A'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Koreatown Mode of Trasnportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(dtown, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Koreatown Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Koreatown Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Koreatown'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Hollywood

In [ ]:
hwood= df.query("Neighborhood == 'Hollywood'")

In [ ]:
hwood.shape

In [ ]:
type(hwood)

In [ ]:
fig = px.bar(hwood, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Hollywood Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(hwood, 
             y='geoid', 
             x="Median Income",
             title='Hollywood Median Income',
             color_discrete_sequence=['#B6E880'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(hwood, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Hollywood Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(hwood, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Hollywood Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Hollywood Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Hollywood'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Pico-Union

In [ ]:
pu= df.query("Neighborhood == 'Pico-Union'")

In [ ]:
pu.shape

In [ ]:
type(pu)

In [ ]:
fig = px.bar(pu, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Pico-Union Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(pu, 
             y='geoid', 
             x="Median Income",
             title='Pico-Union Median Income',
             color_discrete_sequence=['#19D3F3'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(pu, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Pico-Union Mode of Trasnportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(pu, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Pico-Union Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Pico-Union'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Pico-Union Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

## San Fernando Valley

### Panaroma City

In [ ]:
paci= df.query("Neighborhood == 'Panorama City'")

In [ ]:
paci.shape

In [ ]:
type(paci)

In [ ]:
fig = px.bar(paci, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Panorama City Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(paci, 
             y='geoid', 
             x="Median Income",
             title='Panorama City Median Income',
             color_discrete_sequence=['#FF6692'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(paci, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Downtown Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(paci, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Panorama City Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Panorama City'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Panorama City Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Westlake'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Valley Village

In [ ]:
vv= df.query("Neighborhood == 'Valley Village'")

In [ ]:
vv.shape

In [ ]:
type(vv)

In [ ]:
fig = px.bar(vv, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Valley Village Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(vv, 
             y='geoid', 
             x="Median Income",
             title='Valley Village Median Income',
             color_discrete_sequence=['#AB63FA'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(vv, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Valley Village Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(vv, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Valley Village Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Valley Village Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Valley Village'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### North Hollywood

In [ ]:
nhwood= df.query("Neighborhood == 'North Hollywood'")

In [ ]:
nhwood.shape

In [ ]:
type(nhwood)

In [ ]:
fig = px.bar(nhwood, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='North Hollywood Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(nhwood, 
             y='geoid', 
             x="Median Income",
             title='North Hollywood Median Income',
             color_discrete_sequence=['#EF553B'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(nhwood, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='North Hollywood Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(nhwood, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='North Hollywood Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('North Hollywood Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'North Hollywood'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

### Studio City

In [ ]:
stci= df.query("Neighborhood == 'Studio City'")

In [ ]:
stci.shape

In [ ]:
type(stci)

In [ ]:
fig = px.bar(stci, 
             x="geoid", 
             y=["%Hispanic or Latino","%White","%Black",'%Asian','%Native','%Pacific Islander'],
             title='Studio City Racial/Ethnic Breakdown',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Race/Ethnicity"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(stci, 
             y='geoid', 
             x="Median Income",
             title='Studio City Median Income',
             color_discrete_sequence=['#0099C6'],
             orientation='h',
             labels={
                     "geoid": "Census Tracts",
                     "Median Income": "Median Income",
                                      })
fig.show()

In [ ]:
fig = px.bar(stci, 
             x="geoid", 
             y=["%Car Total","%Public transportation",],
             title='Studio City Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
fig = px.bar(stci, 
             x="geoid", 
             y=["%Car Total","%Bus","%Subway or elevated il",'%Long-distance train or commuter rail','%Light rail','%Worked from home'],
             title='Studio City Mode of Transportation',
             labels={
                     "geoid": "Census Tracts",
                     "value": "Percentage",
                     "variable": "Mode of Transportation"
                 },
             )
fig.show()

In [ ]:
##next week, work on bins
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

# name each subplot
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axs

m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax1, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax1.set_title('Public Trasnportation')

m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%Car Total',
                 cmap='YlGn',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                  ax=ax2, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax2.set_title('Car Total')


m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%Hispanic or Latino',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax3, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax3.set_title('Percent Hispanic/Latino')


m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%White',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax4, 
                 legend=True, 
                 scheme='NaturalBreaks',)

ax4.set_title('Percent White')


m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%Black',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax5, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax5.set_title('Percent Black')


m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='%Asian',
                 cmap='OrRd',
                 edgecolor='white', 
                 linewidth=0., 
                 alpha=0.75, 
                 ax=ax6, 
                  legend=True, 
                 scheme='NaturalBreaks',)

ax6.set_title('Percent Asian')

fig.suptitle('Studio City Overview')

In [ ]:
m2w_income_race.query("Neighborhood == 'Studio City'").plot(figsize=(12,10),
                 column='Median Income',
                 cmap='YlGnBu',
                 edgecolor='black', 
                 linewidth=0., 
                 alpha=0.75, 
                 scheme='UserDefined',
                classification_kwds=dict(bins=[12760,23700,39450,63100,77300,100490,115950,139140,150000,200000]),
                 legend=True, 
               )

## High Tranist Area query DF

In [ ]:
nbh_pt20.columns.to_list()

In [ ]:
keep=['geoid',
 'name',
 'Neighborhood',
 'Median Income',
 'Total Workers Commuting',
 'Car Total',
 'Drove alone',
 'Carpooled',
 'Public transportation',
 'Bus',
 'Subway or elevated il',
 'Long-distance train or commuter rail',
 'Light rail',
 'Worked from home',
 '%Car Total',
 '%Drove alone',
 '%Carpooled',
 '%Public transportation',
 '%Bus',
 '%Subway or elevated il',
 '%Long-distance train or commuter rail',
 '%Light rail',
 '%Worked from home',
 'Total Pop Race',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific Islander',
 'Hispanic or Latino',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Pacific Islander',
 '%Hispanic or Latino',]

In [ ]:
nbh_pt20_df=nbh_pt20[keep]

In [ ]:
nbh_pt20_df.head(3)

In [ ]:
type(nbh_pt20_df)

I'm not neither sure of how creat a function/loop for specific column rows characteristis nor able to to these plotly graphs with a function/loop. So this is my makeshift attempt to be efficient

In [ ]:
fig = px.bar(nbh_pt20_df, x="Median Income", y="geoid",facet_col="Neighborhood",facet_col_wrap=3)
fig.show()

Each neighborhood has a different set of geoids, that's the problem here. Well need to think this through.

# Maps

In [ ]:
nbh_pt20.plot()

In [ ]:
m2w_income_race[m2w_income_race['%Public transportation'] > 20].plot(figsize=(12,10),
                 column='%Public transportation',
                 cmap='cividis',
                 legend=True, 
                 scheme='NaturalBreaks',)